In [251]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome()
import time
def toggle_javascript_state(driver, enable=True):
    if enable:
        driver.execute_script("document.documentElement.style.setProperty('display', 'block', 'important');")
    else:
        driver.execute_script("document.documentElement.style.setProperty('display', 'none', 'important');")
    # Refresh the page to apply changes
    driver.refresh()
toggle_javascript_state(driver, enable=True)
main_data = pd.DataFrame(columns=['सि.नं.', 'मतदाता नं', 'मतदाताको नाम', 'उमेर (वर्ष)', 'लिङ्ग',
       'पति/पत्नीको नाम', 'पिता/माताको नाम', 'विवरण','ward','centre'])
driver.get('https://voterlist.election.gov.np/bbvrs1/index_2.php')

In [253]:
# def get_voters(province,district,municipalit,ward,center):

time.sleep(.5)
def select_mun():
    # driver.refresh()
    time.sleep(.5)
    prov_drop_down = Select(driver.find_element(By.ID,'state'))
    prov_drop_down.select_by_index(4)
    time.sleep(.5)
    distric_drop_down = Select(driver.find_element(By.ID,'district'))
    distric_drop_down.select_by_index(3)
    time.sleep(.5)
    mun_drop_down = Select(driver.find_element(By.ID,'vdc_mun'))
    mun_drop_down.select_by_index(6)
    time.sleep(.5)
    ward_drop_down = Select(driver.find_element(By.ID,'ward'))
    return ward_drop_down


driver.execute_cdp_cmd("Emulation.setScriptExecutionDisabled", {"value": False})
driver.refresh()
prov_drop_down = Select(driver.find_element(By.ID,'state'))
prov_drop_down.select_by_index(4)
time.sleep(.5)
distric_drop_down = Select(driver.find_element(By.ID,'district'))
distric_drop_down.select_by_index(3)
time.sleep(.5)
mun_drop_down = Select(driver.find_element(By.ID,'vdc_mun'))
mun_drop_down.select_by_index(6)
time.sleep(.5)
ward_dd = Select(driver.find_element(By.ID,'ward'))
ward_name = [i.text for i in ward_dd.options[1:]]
for ward in ward_name:
    select_mun()
    ward_dd = Select(driver.find_element(By.ID,'ward'))
    print(ward)
    ward_dd.select_by_visible_text(ward)
    time.sleep(.5)
    centre_dd = Select(driver.find_element(By.ID,'reg_centre'))
    centre_name = [i.text for i in centre_dd.options[1:]]
    for centre in centre_name:
        print(centre)
        if len(centre_name) > 1:
            select_mun()
            ward_dd = Select(driver.find_element(By.ID,'ward'))
            ward_dd.select_by_visible_text(ward)
            centre_dd = Select(driver.find_element(By.ID,'reg_centre'))
            centre_dd.select_by_visible_text(centre)
        else:
            centre_dd = Select(driver.find_element(By.ID,'reg_centre'))
            centre_dd.select_by_visible_text(centre)
        driver.find_element(By.ID,'btnSubmit').click()
        # driver.back()
        driver.execute_cdp_cmd("Emulation.setScriptExecutionDisabled", {"value": True})
        driver.refresh()
        time.sleep(.5)

        # toggle_javascript_state(driver,enable=False)
        # driver.refresh()
        # time.sleep(4)
        df = pd.read_html(driver.page_source)[-1]
        print(df.shape)
        df['ward'] = ward
        df['centre'] = centre
        main_data = main_data.append(df)
        driver.execute_cdp_cmd("Emulation.setScriptExecutionDisabled", {"value": False})
        driver.refresh()
        # # toggle_javascript_state(driver,enable=True)
        driver.back()
        time.sleep(.5)
        driver.refresh()


# main_data.columns = ['sn','id','name','age','sex','spouse','parent','detail','ward','centre']
# main_data.drop(columns = ['detail','sn'],inplace = True)
# main_data['ward'] = main_data['ward'].astype('int32')
# main_data['age'] = main_data['age'].astype('int32')
# main_data['id'] = main_data['id'].astype('int32')
# main_data.replace({'पुरुष':'male','महिला':'female'},inplace=True)
# # main_data.to_csv('bhanu.csv',index = False)




In [25]:
# above 100
import pandas as pd 
import plotly.express as pe
import plotly.graph_objects as go
main_data = pd.read_csv('bhanu.csv')
above_100 = main_data.query('age >= 100')
main_data = main_data.query('age <= 100')
area = {1:5.59,2:12.03,3:9.07,4:18.03,5:19.1,6:16.34,7:20.57,8:11.39,9:7.65,10:14.9,11:16.9,12:16.33,13:16.47}
# pd.set_option('display.max_rows',10)

In [3]:
main_data.shape

(40479, 8)

In [38]:

pe.histogram(main_data,x= 'ward',color_discrete_sequence=['green'])

In [21]:
from plotly.subplots import make_subplots
for ward in main_data.ward.unique():
    male_data = main_data.query('sex == "male" & ward == @ward')
    female_data = main_data.query('sex == "female" & ward == @ward')
    fig = make_subplots(rows=1,cols=2,subplot_titles=[f'male {ward}',f'female {ward}'])
    fig.add_trace(pe.histogram(male_data,x= 'age',nbins=20).data[0])
    fig.add_trace(pe.histogram(female_data,x= 'age',nbins=20).data[0],col=2,row = 1)
    # fig.update_yaxes(range=[0, 300], row=1, col=1)
    # fig.update_yaxes(range=[0, 300], row=1, col=2)
    fig.show()
    fig2= pe.pie(main_data.query('ward == @ward'),'sex',title=f'popn {ward}')
    fig2.show()


23650.0